# Goal: Fairness Metrics Computation

The goal of this notebook is to computes statistical fairness metrics, after the execution of a set of mitigation techniques, given:
- a training dataset (mitigated in case of a pre-processing technique),
- a target variable,
- a sensible attribute.


# Import Libraries





In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [2]:
#import libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_validate,cross_val_score,cross_val_predict,train_test_split,StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
from source.utils.config import *
from source.utils.metrics_utils import *

# Configure the notebook


Modify the variable in the next code cell to configure the notebook, insert in `dataset_list` the name of the dataset.

In [5]:
dataset_list = ['diabetes-women', 'stroke-prediction', 'diabetes-prediction'] #'sepsis', 'aids', 'myocardial-infarction', 'alzheimer-disease',
#options: "diabetes-women", "sepsis", 'aids', "myocardial-infarction", 'alzheimer-disease', "diabetes-prediction", "stroke-prediction"

# Compute metrics

In [6]:
for dataset_name in dataset_list:
  dataset_config = datasets_config[dataset_name]
  ignore_cols = dataset_config['ignore_cols']
  target_variable = dataset_config['target_variable']
  target_variable_labels = dataset_config['target_variable_labels']
  sensible_attributes = dataset_config['sensible_attributes']
  default_mappings = dataset_config.get('default_mappings', {})
  n_splits = dataset_config['n_splits']
  for sensible_attribute in sensible_attributes:
    for mitigation in all_techniques:
      print(dataset_name, sensible_attribute, mitigation)

      if mitigation == 'original':
        dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
        df = pd.read_csv(dataset_path)
        predictions_and_tests = compute_predictions_and_tests(df, sensible_attribute, target_variable, n_splits, models, n_estimators, random_seed)
      else:
        # Load the correct source dataset, considering that pre-processing techniques
        # modify the original dataset, while in- and post- processing do not
        if mitigation in new_dataset_mitigations:
          dataset_path = path_to_project + '/data/mitigated/mitigated-{}-{}-{}.csv'.format(dataset_name, sensible_attribute, mitigation) if IN_COLAB else 'data/mitigated/mitigated-{}-{}.csv'.format(dataset_name, mitigation)
        else:
          dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)

        load_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
        with open(load_path, 'rb') as fp:
            predictions_and_tests = pickle.load(fp)

      df = pd.read_csv(dataset_path)
      #df = df.drop(columns=ignore_cols)
      #feature_cols = df.columns
      #sensible_values = [0, 1]  # 0 is the discriminated group, 1 the privileged one

      performance_metrics = compute_performance_metrics(predictions_and_tests, models, n_splits, mitigation)
      #Save performance metrics
      save_path = path_to_project + '/measurements/performance_metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
      with open(save_path, 'wb') as fp:
          pickle.dump(performance_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)

      final_fairness_metrics = compute_final_fairness_metrics(mitigation, predictions_and_tests, target_variable_labels, models, n_splits)

      # print("DIVISION")
      # for metric in fairness_catalogue:
      #   print(metric)
      #   if mitigation in without_model_mitigations:
      #     print(final_fairness_metrics["division"][metric][0])
      #   else:
      #     for m in models:
      #       print(m, final_fairness_metrics["division"][m][metric][0])
      #   print('---------------')

      # print("SUBTRACTION")
      # for metric in fairness_catalogue:
      #   print(metric)
      #   if mitigation in without_model_mitigations:
      #     print(final_fairness_metrics["subtraction"][metric][0])
      #   else:
      #     for m in models:
      #       print(m, final_fairness_metrics["subtraction"][m][metric][0])
      #   print('---------------')
      #Save the metrics results
      save_path = path_to_project + '/measurements/metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
      with open(save_path, 'wb') as fp:
          pickle.dump(final_fairness_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)



diabetes-women AgeCategory original


  0%|          | 0/6 [00:00<?, ?it/s]

Logistic Regression {'GroupFairness': [0.1834850096287018, 0.017943960188798676], 'PredictiveParity': [0.10414480643928774, 0.03775651729849421], 'PredictiveEquality': [0.15901368870688698, 0.13128915568102523], 'EqualOpportunity': [-0.876688059296755, 0.16159530717239812], 'EqualizedOdds': [0.0813258750592886, 0.04575429623387461], 'ConditionalUseAccuracyEquality': [0.023109445810347323, 0.0766881399199534], 'OverallAccuracyEquality': [0.10536546799811342, 0.03237485088329599], 'TreatmentEquality': [-0.813401187446989, 0.2289611362339803], 'FORParity': [-0.5199999999999999, 0.5900508452667448], 'FN': [-0.8331375560977079, 0.2122758701668953], 'FP': [-0.33909570681528883, 0.23012488121176025]}
Decision Tree {'GroupFairness': [0.19334997442191282, 0.03829176851990698], 'PredictiveParity': [0.11167152794974751, 0.025597985233686588], 'PredictiveEquality': [0.18298505416152472, 0.19901026949625777], 'EqualOpportunity': [-0.5099393771557988, 0.1409020084574949], 'EqualizedOdds': [0.0749613

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logistic Regression {'GroupFairness': [0.08943124195517169, 0.09151893936797656], 'PredictiveParity': [0.15011963930154087, 0.0881593175141956], 'PredictiveEquality': [-0.010842053928083463, 0.20418452334624818], 'EqualOpportunity': [-0.3732360097323601, 0.6016015707183429], 'EqualizedOdds': [-0.0034524718772689146, 0.11153462541776724], 'ConditionalUseAccuracyEquality': [0.15833109008024687, 0.15197909931209053], 'OverallAccuracyEquality': [0.1534903378124693, 0.08535729854585236], 'TreatmentEquality': [-0.35339233038348083, 0.5976328379583797], 'FORParity': [-0.22882882882882885, 0.6318546679438901], 'FN': [-0.3491374513077351, 0.5971219608550257], 'FP': [-0.45552880483198754, 0.2720095171888627]}
Decision Tree {'GroupFairness': [0.14964158447934717, 0.051297931801613], 'PredictiveParity': [0.13547263260911202, 0.042802984099196625], 'PredictiveEquality': [0.10400874163607379, 0.06720721183551637], 'EqualOpportunity': [-0.3343651195330451, 0.22744134499047136], 'EqualizedOdds': [0.06

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

Logistic Regression {'GroupFairness': [0.0, 0.0], 'PredictiveParity': [-0.00034328332307031177, 0.011556080898957183], 'PredictiveEquality': [0.0, 0.0], 'EqualOpportunity': [0.33333333333333337, 5.551115123125783e-17], 'EqualizedOdds': [0.0, 0.0], 'ConditionalUseAccuracyEquality': [0.33333333333333337, 5.551115123125783e-17], 'OverallAccuracyEquality': [-0.00034328332307031177, 0.011556080898957183], 'TreatmentEquality': [0.33333333333333337, 5.551115123125783e-17], 'FORParity': [0.33333333333333337, 5.551115123125783e-17], 'FN': [0.33333333333333337, 5.551115123125783e-17], 'FP': [-0.0003936421736605744, 0.2765967946065422]}
Decision Tree {'GroupFairness': [-0.00035587188612099695, 0.001067615658362991], 'PredictiveParity': [-0.00036763805790137347, 0.011573665366180975], 'PredictiveEquality': [0.0, 0.0], 'EqualOpportunity': [0.33333333333333337, 5.551115123125783e-17], 'EqualizedOdds': [-0.00019047619047619213, 0.0005714285714285764], 'ConditionalUseAccuracyEquality': [0.333333333333

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression {'GroupFairness': [0.019355546770520475, 0.018079467886320807], 'PredictiveParity': [0.02117410842717355, 0.041130458329899026], 'PredictiveEquality': [0.01470919582861264, 0.026009775385077177], 'EqualOpportunity': [-0.12483993114867602, 0.05594960619595022], 'EqualizedOdds': [0.010092340609111022, 0.012062999724529374], 'ConditionalUseAccuracyEquality': [0.017487486979032164, 0.017874921507770616], 'OverallAccuracyEquality': [0.02088174044969317, 0.02964101942308813], 'TreatmentEquality': [-0.08544538123173846, 0.08102733073084264], 'FORParity': [-0.020608651833154645, 0.052485013338255324], 'FN': [-0.10313212483755574, 0.041614805670398845], 'FP': [-0.017276311300953036, 0.05780270720283536]}
Decision Tree {'GroupFairness': [0.01487661175655349, 0.014076350122259725], 'PredictiveParity': [0.01872737928971981, 0.0398060809185705], 'PredictiveEquality': [0.014926143684451049, 0.022029165477902568], 'EqualOpportunity': [-0.016416177244785675, 0.0220775941059203], 'E